# Session 6: Machine Learning for Data Assimilation

In this session, we explore the integration of machine learning techniques within data assimilation frameworks.

We will discuss the following three key topics:

1.  Physical Parameter Estimation
2.  ML + EnKF
3.  Likelihood-Free Ensemble Transport Filtering

##1. Physical Parameter Estimation
Infer unknown physical parameters governing a dynamical system from sparse and noisy observations.

### 1.1 Problem Setting: Joint Estimation of $(\sigma, \rho)$
We consider the inverse problem of estimating the parameters $\theta = (\sigma, \rho)$ in the Lorenz-63 system given a partial observation trajectory. The system dynamics are governed by$$\begin{aligned}
\dot x &= \sigma(y-x), \\
\dot y &= x(\rho-z)-y, \\
\dot z &= xy-\beta z.
\end{aligned}$$The ground truth parameters are set to the standard chaotic regime $\theta^* = (\sigma^*, \rho^*) = (10, 28)$ with fixed $\beta = 8/3$. The system is integrated using a Runge-Kutta 4th order scheme with a time step $dt=0.03$. Observations are restricted to the first state component $x$ at discrete intervals $\Delta t_{\text{obs}} = 0.15$. The dataset $Y^\dagger_{50}$ consists of 50 noisy measurements generated from the ground truth trajectory$$Y^\dagger_{50} = \{y^\dagger_1, \ldots, y^\dagger_{50}\}, \quad y^\dagger_j = x(t_j) + \eta_j, \quad \eta_j \sim \mathcal{N}(0, 1.0).$$We assume a Gaussian prior centered near the chaotic attractor but biased from the truth to challenge the assimilation process$$\theta \sim \mathcal{N}\left( \begin{bmatrix} 9.0 \\ 30.0 \end{bmatrix}, \begin{bmatrix} 4.0 & 0 \\ 0 & 9.0 \end{bmatrix} \right).$$

**Note regarding Model Error:** We assume a perfect model structure (deterministic ODE) where uncertainty arises only from unknown parameters and noisy data.

**Ground Truth by MCMC:** Given a candidate parameter set $\theta = (\sigma, \rho)$ and a fixed initial condition $x_0$, the model generates a deterministic trajectory $v(t; \theta)$. Since the observation noise is independent and Gaussian, the likelihood of observing the dataset $Y^\dagger_{50}$ is:
$$p(Y^\dagger_{50} | \theta) \propto \exp\left( -\frac{1}{2\sigma_{\text{obs}}^2} \sum_{j=1}^{50} \|y^\dagger_j - H(v(t_j; \theta))\|^2 \right)$$
Based on this likelihood, we use the MCMC algorithm (Metropolis-Hastings) to sampling from the ground truth.

In [ ]:
import jax
import jax.numpy as jnp
from jax import random, jit, lax
from jax.scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
import time

# ==========================================
# 1. Colab Form Configuration (Compact)
# ==========================================
# @title Experiment Parameters

# @markdown **Ground Truth**
true_sigma = 10.0  # @param {type:"number"}
true_rho = 28.0  # @param {type:"number"}

# @markdown **Prior Beliefs (Mean / Std)**
prior_mu_sigma = 9.0  # @param {type:"number"}
prior_std_sigma = 2.0  # @param {type:"number"}
prior_mu_rho = 30.0  # @param {type:"number"}
prior_std_rho = 3.0  # @param {type:"number"}

# @markdown **MCMC Settings**
obs_num = 50  # @param {type:"number"}
step_size = 0.15  # @param {type:"number"}

# Constants
DT_INT = 0.03
FIXED_OBS_FREQ = 5
SIGMA_OBS = 1.0
N_SAMPLES_FIXED = 50000

# ==========================================
# 2. System Dynamics & Solvers (JAX)
# ==========================================

@jit
def lorenz63_dynamics(state, params):
    """
    Computes Lorenz-63 time derivative.
    Input: state (x, y, z), params (sigma, rho, beta)
    Output: derivative (dx, dy, dz)
    """
    x, y, z = state
    sigma, rho, beta = params
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return jnp.array([dx, dy, dz])

@jit
def rk4_step(state, params, dt):
    """
    Performs a single RK4 integration step.
    Input: current state, params, dt
    Output: next state
    """
    k1 = lorenz63_dynamics(state, params)
    k2 = lorenz63_dynamics(state + 0.5 * dt * k1, params)
    k3 = lorenz63_dynamics(state + 0.5 * dt * k2, params)
    k4 = lorenz63_dynamics(state + dt * k3, params)
    return state + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

@partial(jit, static_argnums=(4,))
def generate_truth(x0, sigma, rho, beta, total_steps):
    """
    Generates the ground truth trajectory.
    Input: x0, physical params, total_steps
    Output: Full state trajectory (N+1, 3)
    """
    params = jnp.array([sigma, rho, beta])
    def step_fn(carry, _):
        next_state = rk4_step(carry, params, DT_INT)
        return next_state, next_state
    _, traj = lax.scan(step_fn, x0, None, length=total_steps)
    return jnp.concatenate([x0[None, :], traj], axis=0)

# ==========================================
# 3. MCMC Engine
# ==========================================

def log_posterior_point(params, y_obs, obs_idx_jax, prior_mu, prior_std, x0_fixed, sim_length):
    """
    Calculates unnormalized log posterior for a parameter set.
    Input: params (sigma, rho), observations, priors, fixed x0
    Output: log_probability scalar
    """
    sigma, rho = params
    # Prior
    lp_sigma = norm.logpdf(sigma, loc=prior_mu[0], scale=prior_std[0])
    lp_rho   = norm.logpdf(rho,   loc=prior_mu[1], scale=prior_std[1])

    # Likelihood (Simulation)
    def sim_func(carry, _):
        ns = rk4_step(carry, jnp.array([sigma, rho, 8.0/3.0]), DT_INT)
        return ns, ns

    _, traj_internal = lax.scan(sim_func, x0_fixed, None, length=sim_length)
    full_traj = jnp.concatenate([x0_fixed[None, :], traj_internal], axis=0)

    y_model = full_traj[obs_idx_jax, 0]
    sse = jnp.sum((y_obs - y_model)**2)
    log_lik = -0.5 * sse / (SIGMA_OBS**2)

    return lp_sigma + lp_rho + log_lik

@partial(jit, static_argnums=(6,))
def run_mcmc_engine(
    key, start_params, y_obs, obs_idx_jax, prior_mu, prior_std,
    sim_length, x0_fixed, rw_step_size
):
    """
    Runs the Metropolis-Hastings MCMC chain.
    Input: PRNGKey, start params, data, priors, simulation specs
    Output: (chain samples, acceptance count)
    """
    init_lp = log_posterior_point(start_params, y_obs, obs_idx_jax, prior_mu, prior_std, x0_fixed, sim_length)

    def mcmc_kernel(carry, key_step):
        curr_params, curr_lp, acc = carry
        noise = random.normal(key_step, (2,)) * rw_step_size
        prop_params = curr_params + noise
        prop_lp = log_posterior_point(prop_params, y_obs, obs_idx_jax, prior_mu, prior_std, x0_fixed, sim_length)

        log_ratio = prop_lp - curr_lp
        accept = jnp.log(random.uniform(key_step)) < log_ratio

        new_params = jnp.where(accept, prop_params, curr_params)
        new_lp = jnp.where(accept, prop_lp, curr_lp)
        new_acc = acc + accept.astype(int)
        return (new_params, new_lp, new_acc), new_params

    keys = random.split(key, N_SAMPLES_FIXED)
    (_, _, total_acc), chain = lax.scan(mcmc_kernel, (start_params, init_lp, 0), keys)
    return chain, total_acc

# ==========================================
# 4. Orchestration & Plotting
# ==========================================

def plot_results(chain, p_mu_s, p_std_s, p_mu_r, p_std_r, t_sigma, t_rho, acc, n_obs):
    """
    Visualizes the posterior density and ground truth.
    """
    plt.figure(figsize=(10, 6))

    # Prior Contours
    x_range = np.linspace(p_mu_s - 3*p_std_s, p_mu_s + 3*p_std_s, 100)
    y_range = np.linspace(p_mu_r - 3*p_std_r, p_mu_r + 3*p_std_r, 100)
    X, Y = np.meshgrid(x_range, y_range)
    Z = (1/(2*np.pi*p_std_s*p_std_r)) * np.exp(-0.5*(((X-p_mu_s)/p_std_s)**2 + ((Y-p_mu_r)/p_std_r)**2))

    custom_levels = Z.max() * np.array([0.01, 0.05, 0.1, 0.3, 0.6, 0.8, 0.9, 0.95, 0.99])
    plt.contour(X, Y, Z, levels=custom_levels, colors='orange', linestyles='--', alpha=0.6)

    # Posterior Scatter (Downsampled)
    step = max(1, len(chain) // 2000)
    plot_chain = chain[::step]
    plt.scatter(plot_chain[:, 0], plot_chain[:, 1],
                color='#1f77b4', s=5, alpha=0.3, label='Posterior (Samples)')

    # Markers
    plt.scatter(t_sigma, t_rho, c='red', marker='*', s=300, zorder=10, label=f'Truth ({t_sigma:.1f}, {t_rho:.1f})')
    plt.scatter(p_mu_s, p_mu_r, c='orange', marker='x', s=150, zorder=10, lw=3, label='Prior Mean')

    plt.title(f"Joint Estimation (MCMC Steps: {N_SAMPLES_FIXED}, Accept: {acc:.1%})\nObs: {n_obs} points")
    plt.xlabel('Sigma'); plt.ylabel('Rho')

    # Dynamic Zoom
    all_x = [t_sigma, p_mu_s, chain[:,0].mean()]
    all_y = [t_rho, p_mu_r, chain[:,1].mean()]
    center_x, center_y = np.mean(all_x), np.mean(all_y)
    span_x = max(np.max(all_x) - np.min(all_x), 4.0) * 0.8 + 2.0
    span_y = max(np.max(all_y) - np.min(all_y), 10.0) * 0.8 + 5.0

    plt.xlim(center_x - span_x, center_x + span_x)
    plt.ylim(center_y - span_y, center_y + span_y)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='upper left')
    plt.show()

def run_experiment_logic():
    """
    Main driver function using global form variables.
    Executes simulation, MCMC, data saving, and plotting.
    """
    t0 = time.time()

    # 1. Generate Truth
    total_steps = int(obs_num * FIXED_OBS_FREQ)
    key = random.PRNGKey(42)
    k1, k2, k3 = random.split(key, 3)

    # Burn-in for initial condition
    x0 = random.normal(k1, (3,))
    for _ in range(100):
        x0 = rk4_step(x0, jnp.array([true_sigma, true_rho, 8.0/3.0]), DT_INT)

    truth_traj = generate_truth(x0, true_sigma, true_rho, 8.0/3.0, total_steps)

    # 2. Observations
    obs_indices = np.arange(0, total_steps + 1, FIXED_OBS_FREQ)
    obs_indices_jax = jnp.array(obs_indices)
    y_truth = truth_traj[obs_indices_jax, 0]
    y_obs = y_truth + random.normal(k2, y_truth.shape) * SIGMA_OBS

    # 3. MCMC Setup
    prior_mu = jnp.array([prior_mu_sigma, prior_mu_rho])
    prior_std = jnp.array([prior_std_sigma, prior_std_rho])
    rw_step_val = jnp.array([step_size/2.0, step_size])
    sim_len_int = int(obs_indices[-1])

    # 4. Run Chain
    # Run the compiled JAX MCMC engine
    chain, total_acc = run_mcmc_engine(
        k3, prior_mu, y_obs, obs_indices_jax,
        prior_mu, prior_std, sim_len_int, x0, rw_step_val
    )

    # 5. Process, Save & Plot
    chain = np.array(chain)
    t_calc = time.time() - t0
    burn_in = int(N_SAMPLES_FIXED * 0.1)

    # Filter burn-in to get effective samples
    clean_chain = chain[burn_in:]
    acc_rate = total_acc / N_SAMPLES_FIXED

    # Save to file
    file_path = 'mcmc_chain.npy'
    np.save(file_path, clean_chain)

    print(f"JAX Calculation: {t_calc:.3f}s")
    print(f"Effective samples (post burn-in): {clean_chain.shape[0]}")
    print(f"Chain saved to: {file_path}")

    plot_results(clean_chain, prior_mu_sigma, prior_std_sigma, prior_mu_rho, prior_std_rho,
                 true_sigma, true_rho, acc_rate, obs_num)

# Execute
if __name__ == "__main__":
    run_experiment_logic()

### 1.2 State Augmentation with EnKF

In this section, we solve the inverse problem using **State Augmentation** within the Ensemble Kalman Filter (EnKF).

### 1.2.1 The Concept
We treat the unknown parameters $\theta = (\sigma, \rho)$ as dynamical variables with zero time derivatives (subject to small random perturbations). We augment the physical state $\mathbf{v} = [x, y, z]^T$ with these parameters to form an augmented state vector $\mathbf{u} \in \mathbb{R}^5$:

$$
\mathbf{u}_j = [\mathbf{v}_j, \theta_j ]^\top = [x_j, y_j, z_j, \sigma_j, \rho_j]^\top
$$

### 1.2.2 Dynamics and Jittering
The evolution of this augmented system combines the physical ODE solver (Runge-Kutta 4) and a random walk model for the parameters (to prevent variance collapse):

$$
\begin{aligned}
\mathbf{v}_{k+1} &= \text{RK4}(\mathbf{v}_k, \theta_k, \Delta t) \\
\theta_{k+1} &= \theta_k + \xi_k, \quad \xi_k \sim \mathcal{N}(0, \Sigma_{\text{jitter}})
\end{aligned}
$$

### 1.2.3 Estimation via EnKF
We apply the standard Ensemble Kalman Filter to this augmented system. By updating the full augmented ensemble $\mathbf{u}^{(i)}$ using observations of only the first component $x$, the filter naturally adjusts the unobserved parameters $\theta$.

The **final estimation** is derived from the marginal distribution of the parameter dimensions ($\sigma, \rho$) within the ensemble at the final time step $T$. We interpret the ensemble members $\{\theta_T^{(1)}, \dots, \theta_T^{(N)}\}$ as samples from the approximate posterior distribution.

In [ ]:
import jax
import jax.numpy as jnp
from jax import random, jit, vmap, lax
from jax.scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.lines import Line2D
from functools import partial
import time
import os

# ==========================================
# 1. Colab Form Configuration
# ==========================================
# @title EnKF Experiment Parameters

# @markdown **Ground Truth**
true_sigma = 10.0  # @param {type:"number"}
true_rho = 28.0  # @param {type:"number"}

# @markdown **Prior Beliefs**
prior_mu_sigma = 9.0  # @param {type:"number"}
prior_std_sigma = 2.0  # @param {type:"number"}
prior_mu_rho = 30.0  # @param {type:"number"}
prior_std_rho = 5.0  # @param {type:"number"}

# @markdown **EnKF Settings**
obs_num = 50  # @param {type:"number"}
ensemble_size = 200  # @param {type:"number"}
step_size = 0.15 # @param {type:"number"}

# Constants
DT_INT = 0.03
FIXED_OBS_FREQ = 5
SIGMA_OBS = 1.0

# ==========================================
# 2. System Dynamics (JAX)
# ==========================================

@jit
def lorenz63_dynamics(state, params):
    x, y, z = state
    sigma, rho, beta = params
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return jnp.array([dx, dy, dz])

@jit
def rk4_step(state, params, dt):
    k1 = lorenz63_dynamics(state, params)
    k2 = lorenz63_dynamics(state + 0.5 * dt * k1, params)
    k3 = lorenz63_dynamics(state + 0.5 * dt * k2, params)
    k4 = lorenz63_dynamics(state + dt * k3, params)
    return state + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

# Generator for Truth
@partial(jit, static_argnums=(4,))
def generate_truth(x0, sigma, rho, beta, total_steps):
    params = jnp.array([sigma, rho, beta])
    def step_fn(carry, _):
        next_state = rk4_step(carry, params, DT_INT)
        return next_state, next_state
    _, traj = lax.scan(step_fn, x0, None, length=total_steps)
    return jnp.concatenate([x0[None, :], traj], axis=0)

# ==========================================
# 3. EnKF Implementation
# ==========================================

@jit
def enkf_update(ensemble, y_obs, R_noise_std):
    """Standard Stochastic EnKF Update Step (Augmented State)."""
    N = ensemble.shape[0]
    ens_mean = jnp.mean(ensemble, axis=0)
    A = ensemble - ens_mean
    HX = ensemble[:, 0:1] # Observe x
    Y_A = A[:, 0:1]

    # S = H P H^T + R
    S = (1.0 / (N - 1.0)) * (Y_A.T @ Y_A) + R_noise_std**2

    # Kalman Gain K
    K = (1.0 / (N - 1.0)) * (A.T @ Y_A) * (1.0 / S)

    # Innovation
    innov = y_obs - HX

    # Update
    ensemble_new = ensemble + (innov @ K.T)
    return ensemble_new

# We use static_argnums for ensemble size (arg 6) to avoid shape errors
@partial(jit, static_argnums=(6,))
def run_enkf_cycle(key, x0_ens_init, prior_mu, prior_std, y_obs_seq, total_steps, n_ens):
    k1, k2 = random.split(key)

    # Initial Parameter Ensemble
    sigma_ens = prior_mu[0] + prior_std[0] * random.normal(k1, (n_ens, 1))
    rho_ens   = prior_mu[1] + prior_std[1] * random.normal(k2, (n_ens, 1))

    # Initial State Ensemble
    state_ens = x0_ens_init + 0.1 * random.normal(key, (n_ens, 3))

    # Combine
    ensemble = jnp.hstack([state_ens, sigma_ens, rho_ens])

    param_jitter_std = jnp.array([0.01, 0.01])
    inflation = 1.02

    def body_fn(carry, t):
        ens, key_iter = carry
        key_step, key_jitter = random.split(key_iter)

        # --- 1. Forecast ---
        def integ_step(e_state, _):
            s, r = e_state[3], e_state[4]
            xyz_next = rk4_step(e_state[0:3], jnp.array([s, r, 8.0/3.0]), DT_INT)
            return jnp.concatenate([xyz_next, e_state[3:5]]), None

        def integrate_ensemble(e):
            e_final, _ = lax.scan(integ_step, e, None, length=FIXED_OBS_FREQ)
            return e_final

        ens_forecast = vmap(integrate_ensemble)(ens)

        # Jitter & Inflation
        noise = random.normal(key_jitter, (n_ens, 2)) * param_jitter_std
        ens_forecast = ens_forecast.at[:, 3:5].add(noise)
        ens_mean = jnp.mean(ens_forecast, axis=0)
        ens_forecast = ens_mean + inflation * (ens_forecast - ens_mean)

        # --- 2. Analysis ---
        ens_analysis = enkf_update(ens_forecast, y_obs_seq[t], SIGMA_OBS)
        ens_analysis = jnp.clip(ens_analysis, a_min=0.1, a_max=100.0)

        return (ens_analysis, key_step), ens_analysis

    (final_ens, _), history = lax.scan(body_fn, (ensemble, key), jnp.arange(len(y_obs_seq)))
    return final_ens

# ==========================================
# 4. Energy Distance Metric
# ==========================================

@jit
def compute_energy_distance(sample_a, sample_b):
    """
    Computes Energy Distance D^2(X, Y) = 2E|X-Y| - E|X-X'| - E|Y-Y'|
    """
    def dist_mean(x, y):
        # x: (N, D), y: (M, D)
        # Calculates pairwise distance matrix then takes mean
        diff = x[:, None, :] - y[None, :, :]
        return jnp.mean(jnp.linalg.norm(diff, axis=-1))

    term_xy = dist_mean(sample_a, sample_b) # Cross distance
    term_xx = dist_mean(sample_a, sample_a) # Internal distance X
    term_yy = dist_mean(sample_b, sample_b) # Internal distance Y

    return 2 * term_xy - term_xx - term_yy

# ==========================================
# 5. Visualization (Modified)
# ==========================================

def plot_enkf_results(ensemble, p_mu_s, p_std_s, p_mu_r, p_std_r, t_sigma, t_rho, n_obs, ens_size, ed_val=None, mcmc_samples=None):
    plt.figure(figsize=(10, 6))

    ens_sigma = np.array(ensemble[:, 3])
    ens_rho   = np.array(ensemble[:, 4])

    # --- Contours (Prior) ---
    x_range = np.linspace(p_mu_s - 3*p_std_s, p_mu_s + 3*p_std_s, 100)
    y_range = np.linspace(p_mu_r - 3*p_std_r, p_mu_r + 3*p_std_r, 100)
    X, Y = np.meshgrid(x_range, y_range)
    Z = (1/(2*np.pi*p_std_s*p_std_r)) * np.exp(-0.5*(((X-p_mu_s)/p_std_s)**2 + ((Y-p_mu_r)/p_std_r)**2))

    custom_levels = Z.max() * np.array([0.01, 0.1, 0.5, 0.9])
    plt.contour(X, Y, Z, levels=custom_levels, colors='orange', linestyles='--', alpha=0.6)

    # --- MCMC Reference (Gray) ---
    if mcmc_samples is not None:
        step = max(1, len(mcmc_samples) // 5000)
        plt.scatter(mcmc_samples[::step, 0], mcmc_samples[::step, 1],
                    c='gray', s=5, alpha=0.15, zorder=1)

    # --- EnKF Posterior (Blue) ---
    plt.scatter(ens_sigma, ens_rho, c='blue', s=10, alpha=0.6, zorder=5)

    # --- REMOVED GAUSSIAN FIT (ELLIPSE) ---

    # --- Markers ---
    plt.scatter(t_sigma, t_rho, c='red', marker='*', s=300, zorder=10, label=f'Truth ({t_sigma:.1f}, {t_rho:.1f})')
    plt.scatter(p_mu_s, p_mu_r, c='orange', marker='x', s=150, zorder=10, lw=3, label='Prior Mean')

    # Updated Title
    title_str = f"Method: State Augmentation (EnKF)\nObs: {n_obs} points | Ensemble Size: {ens_size}"
    if ed_val is not None:
        title_str += f" | Energy Dist vs MCMC: {ed_val:.4f}"

    plt.title(title_str)
    plt.xlabel('Sigma'); plt.ylabel('Rho')

    # Smart Limits
    center_x = (t_sigma + p_mu_s)/2
    center_y = (t_rho + p_mu_r)/2
    span_x = max(abs(t_sigma - p_mu_s), 4.0) * 1.5
    span_y = max(abs(t_rho - p_mu_r), 10.0) * 1.5
    plt.xlim(center_x - span_x, center_x + span_x)
    plt.ylim(center_y - span_y, center_y + span_y)

    plt.grid(True, alpha=0.3)

    # --- Custom Legend ---
    legend_elements = [
        Line2D([0], [0], color='orange', linestyle='--', lw=2),
        Line2D([0], [0], color='blue', marker='o', linestyle='None', alpha=0.6),
        # Removed Gaussian Fit Line
        Line2D([0], [0], marker='*', color='w', markerfacecolor='r', markersize=15),
        Line2D([0], [0], marker='x', color='orange', markersize=10, markeredgewidth=2, linestyle='None')
    ]
    legend_labels = ['Prior', 'EnKF Ensemble', 'Truth', 'Prior Mean']

    # Add MCMC to legend if present
    if mcmc_samples is not None:
        legend_elements.insert(1, Line2D([0], [0], color='gray', marker='o', linestyle='None', alpha=0.5))
        legend_labels.insert(1, 'MCMC Samples')

    plt.legend(legend_elements, legend_labels, loc='upper left')

    plt.show()

# ==========================================
# 6. Main Logic
# ==========================================

def run_experiment_logic():
    t0 = time.time()

    # 1. Generate Truth
    total_steps = obs_num * FIXED_OBS_FREQ
    key = random.PRNGKey(42)
    k1, k2, k3 = random.split(key, 3)

    x0 = random.normal(k1, (3,))
    for _ in range(100): x0 = rk4_step(x0, jnp.array([true_sigma, true_rho, 8.0/3.0]), DT_INT)

    truth_traj = generate_truth(x0, true_sigma, true_rho, 8.0/3.0, total_steps)

    # 2. Observations
    obs_indices = np.arange(FIXED_OBS_FREQ, total_steps + 1, FIXED_OBS_FREQ)
    obs_indices_jax = jnp.array(obs_indices)
    y_truth = truth_traj[obs_indices_jax, 0]
    y_obs = y_truth + random.normal(k2, y_truth.shape) * SIGMA_OBS

    # 3. Run EnKF
    prior_mu = jnp.array([prior_mu_sigma, prior_mu_rho])
    prior_std = jnp.array([prior_std_sigma, prior_std_rho])
    x0_ens = jnp.tile(x0, (ensemble_size, 1))

    final_ens = run_enkf_cycle(
        k3, x0_ens, prior_mu, prior_std, y_obs, obs_num, ensemble_size
    )

    t_calc = time.time() - t0
    print(f"EnKF Cycle ({obs_num} obs): {t_calc:.4f}s")

    # 4. Process MCMC for Metric & Plotting
    ed_val = None
    mcmc_params = None

    if os.path.exists('mcmc_chain.npy'):
        mcmc_full = np.load('mcmc_chain.npy')

        # 1. For Metric Calculation (Downsampled)
        enkf_params = final_ens[:, 3:5]

        # Ensure we don't take more samples than available
        n_metric = min(1000, mcmc_full.shape[0])
        idx_metric = np.linspace(0, mcmc_full.shape[0]-1, n_metric).astype(int)
        mcmc_metric_sample = jnp.array(mcmc_full[idx_metric])

        ed_val = compute_energy_distance(enkf_params, mcmc_metric_sample)
        print(f"Energy Distance (EnKF vs MCMC): {ed_val:.5f}")

        # 2. For Plotting
        mcmc_params = mcmc_full
    else:
        print("Warning: 'mcmc_chain.npy' not found. Cannot calculate Energy Distance.")

    # 5. Plot
    plot_enkf_results(
        final_ens,
        prior_mu_sigma, prior_std_sigma,
        prior_mu_rho, prior_std_rho,
        true_sigma, true_rho,
        obs_num,
        ensemble_size,
        ed_val,
        mcmc_samples=mcmc_params
    )

if __name__ == "__main__":
    run_experiment_logic()

### 1.3 Parameter Estimation via Optimization (MAP)

In this section, we solve the inverse problem by formulating it as a variational optimization task. Unlike the EnKF, which updates a distribution of particles, this method seeks the single parameter set $\theta$ that maximizes the posterior probability density.

#### 1.3.1 The Objective Function
We define the Maximum A Posteriori (MAP) estimator $\hat{\theta}_{\text{MAP}}$ as the minimizer of a negative log-posterior cost function $J(\theta)$. This cost function consists of two terms: a data fidelity term (likelihood) and a regularization term (prior).

Given the Gaussian assumptions on observation noise and the prior distribution defined in Section 1.1, the cost function is defined as:

$$
J(\theta) = \underbrace{\frac{1}{2\sigma_{\text{obs}}^2} \sum_{j=1}^{50} \|y^\dagger_j - H(u(t_j; \theta))\|^2}_{\mathcal{L}_{\text{data}}(\theta)} + \underbrace{\frac{1}{2} (\theta - \mu_{\text{prior}})^\top \Sigma_{\text{prior}}^{-1} (\theta - \mu_{\text{prior}})}_{\mathcal{L}_{\text{reg}}(\theta)}
$$

Here, $u(t_j; \theta)$ represents the deterministic solution of the Lorenz-63 ODEs integrated from $t_0$ to $t_j$ using parameters $\theta$. The term $\mathcal{L}_{\text{reg}}$ penalizes deviations from the prior mean $\mu_{\text{prior}} = [9.0, 30.0]^\top$, preventing overfitting to the sparse noise.

#### 1.3.2 Optimization Strategy
We treat the parameter estimation as a non-linear optimization problem to find the global minimum of the cost function:

$$
\hat{\theta}_{\text{MAP}} = \operatorname*{argmin}_{\theta} J(\theta)
$$

We solve this using gradient-based optimization (e.g., L-BFGS-B or Adam). The gradients $\nabla_\theta J$ are computed via Automatic Differentiation (AD) through the Runge-Kutta solver, allowing backpropagation of error signals from the observation times $t_j$ back to the parameters $\theta$.

#### 1.3.3 Uncertainty Quantification via Laplace Approximation
A standard optimization yields only a point estimate $\hat{\theta}_{\text{MAP}}$. To recover the uncertainty (posterior covariance) without the computational cost of MCMC, we employ the **Laplace Approximation**.

We approximate the posterior distribution $p(\theta | Y^\dagger)$ as a Gaussian centered at the MAP estimate:
$$
p(\theta | Y^\dagger) \approx \mathcal{N}(\hat{\theta}_{\text{MAP}}, \Sigma_{\text{post}})
$$
The posterior covariance $\Sigma_{\text{post}}$ is approximated by the inverse of the Hessian (second-order partial derivatives) of the cost function evaluated at the optimum:
$$
\Sigma_{\text{post}} \approx \left( \nabla^2 J(\hat{\theta}_{\text{MAP}}) \right)^{-1}
$$
Intuitively, the Hessian $\mathcal{H} = \nabla^2 J$ measures the curvature of the loss landscape. A sharp valley (large curvature) implies high certainty (small variance), while a flat valley implies high uncertainty (large variance).

In [ ]:
import jax
import jax.numpy as jnp
from jax import random, jit, vmap, lax, value_and_grad, hessian
from jax.scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.lines import Line2D
from functools import partial
import time
import os
import scipy.optimize

# ==========================================
# 1. Colab Form Configuration
# ==========================================
# @title MAP & Laplace Experiment Parameters

# @markdown **Ground Truth**
true_sigma = 10.0  # @param {type:"number"}
true_rho = 28.0  # @param {type:"number"}

# @markdown **Prior Beliefs**
prior_mu_sigma = 9.0  # @param {type:"number"}
prior_std_sigma = 2.0  # @param {type:"number"}
prior_mu_rho = 30.0  # @param {type:"number"}
prior_std_rho = 5.0  # @param {type:"number"}

# @markdown **Optimization Settings**
optimizer_type = "Adam" # @param ["L-BFGS-B", "Adam"]
obs_num = 50  # @param {type:"number"}
learning_rate = 0.01 # @param {type:"number"} (For Adam only)
adam_steps = 2000 # @param {type:"number"} (For Adam only)

# Constants
DT_INT = 0.03
FIXED_OBS_FREQ = 5
SIGMA_OBS = 1.0

# ==========================================
# 2. System Dynamics (JAX)
# ==========================================

@jit
def lorenz63_dynamics(state, params):
    x, y, z = state
    sigma, rho, beta = params
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z
    return jnp.array([dx, dy, dz])

@jit
def rk4_step(state, params, dt):
    k1 = lorenz63_dynamics(state, params)
    k2 = lorenz63_dynamics(state + 0.5 * dt * k1, params)
    k3 = lorenz63_dynamics(state + 0.5 * dt * k2, params)
    k4 = lorenz63_dynamics(state + dt * k3, params)
    return state + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

# Generator for Truth
# total_steps is arg 4, mark as static
@partial(jit, static_argnums=(4,))
def generate_truth(x0, sigma, rho, beta, total_steps):
    params = jnp.array([sigma, rho, beta])
    def step_fn(carry, _):
        next_state = rk4_step(carry, params, DT_INT)
        return next_state, next_state
    _, traj = lax.scan(step_fn, x0, None, length=total_steps)
    return jnp.concatenate([x0[None, :], traj], axis=0)

# ==========================================
# 3. MAP Optimization & Laplace Approximation
# ==========================================

# FIX: Marked total_steps (index 2) as static to satisfy lax.scan requirement
@partial(jit, static_argnums=(2,))
def forward_model(theta, x0, total_steps):
    """Integrates system for given params theta=[sigma, rho]. Returns observations."""
    params = jnp.array([theta[0], theta[1], 8.0/3.0])

    def step(carry, _):
        s = rk4_step(carry, params, DT_INT)
        return s, s

    # Run full trajectory
    _, traj = lax.scan(step, x0, None, length=total_steps)

    # Extract observations at specific indices
    # Since total_steps is static, jnp.arange generates a constant array
    obs_indices = jnp.arange(FIXED_OBS_FREQ - 1, total_steps, FIXED_OBS_FREQ)
    model_obs = traj[obs_indices, 0] # Return only x component
    return model_obs

# cost_function expects total_steps as static (index 5)
@partial(jit, static_argnums=(5,))
def cost_function(theta, x0, y_obs, prior_mu, prior_cov_inv, total_steps):
    """Negative Log Posterior: Data Likelihood + Prior Regularization."""
    # 1. Data Likelihood (MSE)
    # forward_model handles total_steps as static now
    y_pred = forward_model(theta, x0, total_steps)
    diff = y_obs - y_pred
    nll_data = (0.5 / SIGMA_OBS**2) * jnp.sum(diff**2)

    # 2. Prior Regularization (Mahalanobis)
    delta = theta - prior_mu
    nll_prior = 0.5 * (delta @ prior_cov_inv @ delta.T)

    return nll_data + nll_prior

def run_optimization(x0, y_obs, prior_mu, prior_std, obs_num):
    """Solves argmin J(theta) using selected optimizer."""

    prior_cov = jnp.diag(prior_std**2)
    prior_cov_inv = jnp.diag(1.0 / prior_std**2)
    total_steps = obs_num * FIXED_OBS_FREQ

    # JIT the value and gradient function
    # total_steps (arg 5) is static
    loss_and_grad = jit(value_and_grad(cost_function), static_argnums=(5,))

    # Initial Guess (Mean of Prior)
    theta_init = prior_mu

    print(f"Starting Optimization ({optimizer_type})...")

    if optimizer_type == "L-BFGS-B":
        # Wrap JAX function for Scipy
        # We convert inputs to numpy float64 for stability in Scipy
        def scipy_fun(w):
            # w comes from scipy as numpy array
            l, g = loss_and_grad(jnp.array(w), x0, y_obs, prior_mu, prior_cov_inv, total_steps)
            return np.array(l, dtype=np.float64), np.array(g, dtype=np.float64)

        # Removed 'disp' from options to fix DeprecationWarning
        res = scipy.optimize.minimize(
            scipy_fun,
            np.array(theta_init, dtype=np.float64),
            method='L-BFGS-B',
            jac=True,
            options={'maxiter': 100}
        )
        theta_opt = jnp.array(res.x)
        print(f"  Optimization Success: {res.success}")
        print(f"  Iterations: {res.nit}")

    elif optimizer_type == "Adam":
        # Simple Adam Loop
        theta = theta_init
        m = jnp.zeros_like(theta)
        v = jnp.zeros_like(theta)
        beta1, beta2, eps = 0.9, 0.999, 1e-8

        @jit
        def adam_step(i, carry):
            t, m_t, v_t = carry
            l, g = loss_and_grad(t, x0, y_obs, prior_mu, prior_cov_inv, total_steps)

            m_t = beta1 * m_t + (1 - beta1) * g
            v_t = beta2 * v_t + (1 - beta2) * g**2
            m_hat = m_t / (1 - beta1**(i+1))
            v_hat = v_t / (1 - beta2**(i+1))
            t_new = t - learning_rate * m_hat / (jnp.sqrt(v_hat) + eps)
            return (t_new, m_t, v_t)

        final_carry = lax.fori_loop(0, adam_steps, adam_step, (theta, m, v))
        theta_opt = final_carry[0]

    return theta_opt

def compute_laplace_covariance(theta_opt, x0, y_obs, prior_mu, prior_std, obs_num):
    """Computes Inverse Hessian at theta_opt."""
    prior_cov_inv = jnp.diag(1.0 / prior_std**2)
    total_steps = obs_num * FIXED_OBS_FREQ

    # JIT Hessian computation
    H_fn = jit(hessian(cost_function), static_argnums=(5,))
    H = H_fn(theta_opt, x0, y_obs, prior_mu, prior_cov_inv, total_steps)

    # Covariance = Inverse Hessian
    cov_post = jnp.linalg.inv(H)
    return cov_post

# ==========================================
# 4. Energy Distance Metric (Reused)
# ==========================================

@jit
def compute_energy_distance(sample_a, sample_b):
    """D^2(X, Y) = 2E|X-Y| - E|X-X'| - E|Y-Y'|"""
    def dist_mean(x, y):
        diff = x[:, None, :] - y[None, :, :]
        return jnp.mean(jnp.linalg.norm(diff, axis=-1))

    term_xy = dist_mean(sample_a, sample_b)
    term_xx = dist_mean(sample_a, sample_a)
    term_yy = dist_mean(sample_b, sample_b)
    return 2 * term_xy - term_xx - term_yy

# ==========================================
# 5. Visualization
# ==========================================

def plot_map_results(theta_opt, laplace_samples, p_mu, p_std, t_sigma, t_rho, n_obs, ed_val=None, mcmc_samples=None):
    plt.figure(figsize=(10, 6))

    p_mu_s, p_mu_r = p_mu[0], p_mu[1]
    p_std_s, p_std_r = p_std[0], p_std[1]

    # --- Contours (Prior) ---
    x_range = np.linspace(p_mu_s - 3*p_std_s, p_mu_s + 3*p_std_s, 100)
    y_range = np.linspace(p_mu_r - 3*p_std_r, p_mu_r + 3*p_std_r, 100)
    X, Y = np.meshgrid(x_range, y_range)
    Z = (1/(2*np.pi*p_std_s*p_std_r)) * np.exp(-0.5*(((X-p_mu_s)/p_std_s)**2 + ((Y-p_mu_r)/p_std_r)**2))
    plt.contour(X, Y, Z, levels=Z.max() * np.array([0.01, 0.1, 0.5, 0.9]), colors='orange', linestyles='--', alpha=0.6)

    # --- MCMC Reference (Gray) ---
    if mcmc_samples is not None:
        step = max(1, len(mcmc_samples) // 5000)
        plt.scatter(mcmc_samples[::step, 0], mcmc_samples[::step, 1], c='gray', s=5, alpha=0.15, zorder=1)

    # --- Laplace Samples (Blue) ---
    plt.scatter(laplace_samples[:, 0], laplace_samples[:, 1], c='blue', s=10, alpha=0.3, zorder=5, label='Laplace Samples')

    # --- MAP Point Estimate (Green Star) ---
    plt.scatter(theta_opt[0], theta_opt[1], c='lime', marker='*', s=400, edgecolors='black', zorder=20, label='MAP Estimate')

    # --- Markers ---
    plt.scatter(t_sigma, t_rho, c='red', marker='*', s=300, zorder=10, label=f'Truth ({t_sigma:.1f}, {t_rho:.1f})')
    plt.scatter(p_mu_s, p_mu_r, c='orange', marker='x', s=150, zorder=10, lw=3, label='Prior Mean')

    # Title
    title_str = f"Method: MAP Optimization ({optimizer_type}) + Laplace Approx\nObs: {n_obs} points"
    if ed_val is not None:
        title_str += f" | Energy Dist vs MCMC: {ed_val:.4f}"
    plt.title(title_str)
    plt.xlabel('Sigma'); plt.ylabel('Rho')

    # Limits
    center_x = (t_sigma + p_mu_s)/2
    center_y = (t_rho + p_mu_r)/2
    span_x = max(abs(t_sigma - p_mu_s), 4.0) * 1.5
    span_y = max(abs(t_rho - p_mu_r), 10.0) * 1.5
    plt.xlim(center_x - span_x, center_x + span_x)
    plt.ylim(center_y - span_y, center_y + span_y)
    plt.grid(True, alpha=0.3)

    # Legend
    legend_elements = [
        Line2D([0], [0], color='orange', linestyle='--', lw=2),
        Line2D([0], [0], color='blue', marker='o', linestyle='None', alpha=0.6),
        Line2D([0], [0], marker='*', color='lime', markersize=15, markeredgecolor='black', linestyle='None'),
        Line2D([0], [0], marker='*', color='w', markerfacecolor='r', markersize=15),
    ]
    legend_labels = ['Prior', 'Laplace Approx', 'MAP Estimate', 'Truth']

    if mcmc_samples is not None:
        legend_elements.insert(1, Line2D([0], [0], color='gray', marker='o', linestyle='None', alpha=0.5))
        legend_labels.insert(1, 'MCMC Samples')

    plt.legend(legend_elements, legend_labels, loc='upper left')
    plt.show()

# ==========================================
# 6. Main Logic
# ==========================================

def run_experiment_logic():
    t0 = time.time()

    # 1. Generate Truth (Identical seed to EnKF)
    total_steps = obs_num * FIXED_OBS_FREQ
    key = random.PRNGKey(42)
    k1, k2, k3 = random.split(key, 3)

    x0 = random.normal(k1, (3,))
    # Spin up
    for _ in range(100): x0 = rk4_step(x0, jnp.array([true_sigma, true_rho, 8.0/3.0]), DT_INT)

    truth_traj = generate_truth(x0, true_sigma, true_rho, 8.0/3.0, total_steps)

    # 2. Observations
    obs_indices = np.arange(FIXED_OBS_FREQ, total_steps + 1, FIXED_OBS_FREQ)
    y_truth = truth_traj[jnp.array(obs_indices), 0]
    y_obs = y_truth + random.normal(k2, y_truth.shape) * SIGMA_OBS

    # 3. Optimization (MAP)
    prior_mu = jnp.array([prior_mu_sigma, prior_mu_rho])
    prior_std = jnp.array([prior_std_sigma, prior_std_rho])

    theta_opt = run_optimization(x0, y_obs, prior_mu, prior_std, obs_num)

    # 4. Uncertainty (Laplace Approximation)
    cov_post = compute_laplace_covariance(theta_opt, x0, y_obs, prior_mu, prior_std, obs_num)

    print(f"MAP Estimate: sigma={theta_opt[0]:.2f}, rho={theta_opt[1]:.2f}")
    print(f"Time Elapsed: {time.time() - t0:.4f}s")

    # 5. Sample from Laplace for Metrics & Plotting
    # Sample 1000 points from N(theta_opt, cov_post)
    try:
        laplace_samples = random.multivariate_normal(k3, theta_opt, cov_post, shape=(1000,))
    except ValueError:
        print("Warning: Covariance matrix is not positive definite. Falling back to diagonal approximation.")
        laplace_samples = random.multivariate_normal(k3, theta_opt, jnp.diag(jnp.diag(cov_post)), shape=(1000,))

    # 6. Compare with MCMC (Energy Distance)
    ed_val = None
    mcmc_params = None

    if os.path.exists('mcmc_chain.npy'):
        mcmc_full = np.load('mcmc_chain.npy')

        # Downsample MCMC for metric
        n_metric = min(1000, mcmc_full.shape[0])
        idx_metric = np.linspace(0, mcmc_full.shape[0]-1, n_metric).astype(int)
        mcmc_metric_sample = jnp.array(mcmc_full[idx_metric])

        ed_val = compute_energy_distance(laplace_samples, mcmc_metric_sample)
        print(f"Energy Distance (Laplace vs MCMC): {ed_val:.5f}")

        mcmc_params = mcmc_full
    else:
        print("Warning: 'mcmc_chain.npy' not found. Cannot calculate Energy Distance.")

    # 7. Plot
    plot_map_results(
        theta_opt,
        laplace_samples,
        prior_mu, prior_std,
        true_sigma, true_rho,
        obs_num,
        ed_val,
        mcmc_samples=mcmc_params
    )

if __name__ == "__main__":
    run_experiment_logic()

## 2. ML + KF

In this section, we transition to a high-dimensional chaotic system to evaluate the synergy between Machine Learning (ML) and Ensemble Kalman Filtering (EnKF). We consider the Lorenz-96 model, a standard benchmark for atmospheric dynamics and data assimilation.

### 2.1 Problem Setting: Lorenz-96 System

We consider a 40-dimensional Lorenz-96 system, where the state vector $\mathbf{v} \in \mathbb{R}^{40}$ evolves according to the following set of coupled ordinary differential equations:

$$
\frac{d v_k}{d t} = (v_{k+1} - v_{k-2}) v_{k-1} - v_k + F, \quad k = 1, \dots, 40,
$$

with cyclic boundary conditions ($v_{-1} = v_{39}, v_0 = v_{40}, v_{41} = v_1$). The forcing term is fixed at $F=8$ to ensure fully chaotic behavior.

**Simulation and Observation Details:**
* **Dynamic Noise** Denote the discrete Lorenz-96 dynamics by $\Psi$. Then the forward dynamic is given by:
$$v_{j+1} = \Psi(v_j) + \xi_j, \quad \xi_j\sim \mathcal{N}(0,\Sigma),$$
where $\Sigma = \sigma_v^2 I$ and we choose $\sigma_v = 0.01$.
* **Temporal Discretization:** The system is integrated using a 4th-order Runge-Kutta (RK4) scheme with an internal integration step of $\delta t = 0.01$.
* **Assimilation Window:** Observations are available every $\Delta t_{\text{obs}} = 0.05$, corresponding to 5 RK4 integration steps per assimilation cycle.
* **Full Observations:** We consider a fully observed scenario where the entire state vector is measured. The observation vector $\mathbf{y} \in \mathbb{R}^{40}$ is given by:
$$\mathbf{y}_j = \mathbf{v}_j + \eta_j, \quad \eta_j \sim \mathcal{N}(0, \Gamma)$$
where the observation operator $h(\mathbf{v})$ is the identity map. We choose $\Gamma = \sigma_y^2 \mathbf{I}$ with $\sigma_y = 1$.

### 2.2 Baseline: The Perturbed-Observation Ensemble Kalman Filter (EnKF)

As a baseline for high-dimensional assimilation, we employ the **Perturbed-Observation EnKF**. This stochastic variant accounts for both the uncertainty in the forward dynamics and the statistical properties of the observation noise by treating the observations as random variables.

#### 2.2.1 Forecast Step (with Dynamic Noise)
Each ensemble member $\mathbf{v}_{j-1}^{(n)}$ is evolved forward to time $t_j$ using the Lorenz-96 dynamics $\Psi$. To account for model uncertainty, we inject additive dynamic noise $\xi_j^{(n)}$:

$$
\hat{\mathbf{v}}_j^{(n)} = \Psi(\mathbf{v}_{j-1}^{(n)}) + \xi_j^{(n)}, \quad \xi_j^{(n)} \sim \mathcal{N}(0, \Sigma)
$$

where $\Sigma = \sigma_v^2 \mathbf{I}$ is the dynamic noise covariance. The predictive ensemble mean in state space and its projection into observation space are defined as:

$$
\bar{\mathbf{v}}_j = \frac{1}{N} \sum_{n=1}^N \hat{\mathbf{v}}_j^{(n)}, \quad \bar{h}_j = \frac{1}{N} \sum_{n=1}^N h(\hat{\mathbf{v}}_j^{(n)})
$$

#### 2.2.2 Analysis Step
When a sparse observation $\mathbf{y}^\dagger_j$ is received, the filter updates each forecast member. Following the perturbed-observation approach, each member incorporates a realization of the observation noise $\eta_j^{(n)} \sim \mathcal{N}(0, \Gamma)$, where $\Gamma = \sigma_y^2 \mathbf{I}$.

The update is governed by the empirical cross-covariance matrices. We denote the state-observation covariance $\hat{\mathbf{C}}^{vh}_j$ and the innovation covariance $\hat{\mathbf{C}}^{hh}_j$ using the tensor product $\otimes$ as follows:

$$
\hat{\mathbf{C}}^{vh}_j = \frac{1}{N-1} \sum_{n=1}^N (\hat{\mathbf{v}}^{(n)}_j - \bar{\mathbf{v}}_j) \otimes (h(\hat{\mathbf{v}}^{(n)}_j) - \bar{h}_j)
$$

$$
\hat{\mathbf{C}}^{hh}_j = \frac{1}{N-1} \sum_{n=1}^N (h(\hat{\mathbf{v}}^{(n)}_j) - \bar{h}_j) \otimes (h(\hat{\mathbf{v}}^{(n)}_j) - \bar{h}_j)
$$

The analysis ensemble $\mathbf{v}_j^{(n)}$ is then computed by correcting the forecast with the noisy observation residual:

$$
\mathbf{v}_j^{(n)} = \hat{\mathbf{v}}_j^{(n)} + \hat{\mathbf{C}}^{vh}_j \left( \hat{\mathbf{C}}^{hh}_j + \Gamma \right)^{-1} \left( \mathbf{y}^\dagger_j - h(\hat{\mathbf{v}}_j^{(n)}) - \eta_j^{(n)} \right)
$$

#### 2.2.3 Regularization: Inflation and Localization

In high-dimensional systems like Lorenz-96, using a limited ensemble size ($N \ll 40$) typically leads to **filter divergence** due to systematic underestimation of error covariances and the appearance of spurious long-range correlations. We address these issues using distance-dependent localization and post-analysis multiplicative inflation.

#### Covariance Localization via Gaspari-Cohn (GC) Function
To eliminate spurious correlations between physically distant variables, we regularize the empirical covariance matrices using a tapering function. The state-observation covariance $\hat{\mathbf{C}}^{vh}_j$ is modified by an element-wise (Schur) product with a localization matrix $\mathbf{L}$:

$$
\hat{\mathbf{C}}^{vh}_{j, \text{loc}} = \mathbf{L} \circ \hat{\mathbf{C}}^{vh}_j
$$

The elements of $\mathbf{L}$ are defined by the **Gaspari-Cohn (GC) function** $C_0(d, r)$, a compactly supported $5^{\text{th}}$-order polynomial that mimics a Gaussian distribution:



$$
L_{ki} = C_0(d(k, i), r) =
\begin{cases}
1 - \frac{5}{3}x^2 + \frac{5}{8}x^3 + \frac{1}{2}x^4 - \frac{1}{4}x^5, & 0 \leq d < r \\
4 - 5x + \frac{5}{3}x^2 + \frac{5}{8}x^3 - \frac{1}{2}x^4 + \frac{1}{12}x^5 - \frac{2}{3x}, & r \leq d < 2r \\
0, & d \geq 2r
\end{cases}
$$

where $r$ is the localization radius. This ensures that observations only influence state variables within a local neighborhood.

#### Post-Analysis Multiplicative Inflation
To compensate for the loss of variance caused by the linear update and sampling errors, we apply a scalar inflation factor $\alpha > 1$ to the **analysis ensemble** $\mathbf{v}_j^{(n)}$ after the update is completed:

$$
\mathbf{v}_j^{(n)} \leftarrow \bar{\mathbf{v}}_j^{\text{ana}} + \alpha (\mathbf{v}_j^{(n)} - \bar{\mathbf{v}}_j^{\text{ana}})
$$

where $\bar{\mathbf{v}}_j^{\text{ana}} = \frac{1}{N} \sum_{n=1}^N \mathbf{v}_j^{(n)}$ is the mean of the updated ensemble. This post-processing step ensures that the ensemble spread remains sufficiently large to represent the posterior uncertainty and avoids filter collapse in subsequent cycles.

In [ ]:
import jax
import jax.numpy as jnp
from jax import random, jit, lax
from functools import partial
import matplotlib.pyplot as plt

# --- Simulation Parameters (Colab Forms) ---
n = 40  # @param {type:"integer"}
F = 8.0  # @param {type:"number"}
dt_integration = 0.01  # @param {type:"number"}
dt_obs = 0.05  # @param {type:"number"}
num_cycles = 200  # @param {type:"integer"}
sigma_v = 0.01  # @param {type:"number"}
sigma_y = 1  # @param {type:"number"}

# --- Derived Parameters ---
m = n
steps_per_obs = int(dt_obs / dt_integration)
Q = jnp.eye(n) * (sigma_v ** 2)
R_cov = jnp.eye(m) * (sigma_y ** 2)

# --- Dynamics & Integration ---
@jit
def rk4_step(x, dt):
    """
    Performs a single RK4 integration step.
    Input: x (state), dt (time step)
    Output: Updated state
    """
    f = lambda y: (jnp.roll(y, 1) - jnp.roll(y, -2)) * jnp.roll(y, -1) - y + F
    k1 = dt * f(x)
    k2 = dt * f(x + k1/2)
    k3 = dt * f(x + k2/2)
    k4 = dt * f(x + k3)
    return x + (k1 + 2 * k2 + 2 * k3 + k4) / 6.0

@partial(jit, static_argnums=(1,))
def integrate_window(x, steps):
    """
    Integrates system forward for 'steps' iterations.
    Input: x (initial state), steps (int)
    Output: State after integration window
    """
    def body_fun(i, val):
        return rk4_step(val, dt_integration)
    return lax.fori_loop(0, steps, body_fun, x)

@jit
def simulation_step(carry, _):
    """
    Executes one assimilation cycle (Full Observations).
    Input: carry (rng_key, current_state), _ (ignored)
    Output: (new_carry), (true_state, observation)
    """
    key, x = carry
    key, subkey_q, subkey_r = random.split(key, 3)

    x_det = integrate_window(x, steps_per_obs)

    process_noise = random.multivariate_normal(subkey_q, jnp.zeros(n), Q)
    x_next = x_det + process_noise

    obs_noise = random.multivariate_normal(subkey_r, jnp.zeros(n), R_cov)
    y_full = x_next + obs_noise

    return (key, x_next), (x_next, y_full)

# --- Execution ---
key = random.PRNGKey(42)
key, subkey = random.split(key)

# Initialize and Burn-in
x0 = random.normal(subkey, (n,))
x0 = integrate_window(x0, 1000)

# Run Simulation
initial_carry = (key, x0)
_, (true_states_history, observations_history) = lax.scan(simulation_step, initial_carry, None, length=num_cycles)

# Append initial state
all_true_states = jnp.vstack([x0, true_states_history])

# --- Visualization: System Evolution ---
plt.figure(figsize=(12, 4))
plt.imshow(all_true_states.T, aspect='auto', cmap='coolwarm', interpolation='nearest', vmin=-12, vmax=12)
plt.ylabel('State Index (0-39)')
plt.xlabel('Assimilation Cycle')
plt.title('True States (Full System)')
plt.colorbar(label='State Value')
plt.tight_layout()
plt.grid(False)
plt.show()

### 2.3 Variational Inference

Consider the filtering distributions for a dynamical system at times $j = 0,\dots,J$:
$$\pi_j = \mathbb{P}(v_j | y_1^\dagger, \dots, y_j^\dagger)$$

Filtering evolves according to the recursion:\
&nbsp; **Prediction step**: $\widehat\pi_{j+1} = \mathsf{P}\pi_j$, where $\widehat\pi_{j+1} = \mathbb{P}(v_{j+1}|y_1^\dagger, \dots, y_j^\dagger)$\
&nbsp; **Analysis step**: $\pi_{j+1} = \mathsf{A}(\widehat\pi_{j+1};y_{j+1}^\dagger)$

Approximate $\pi_{j+1} \approx q_{j+1}$ at each step of filtering by minmizing the KL divergence objective:
$$\arg\min_q D_{KL}(q || \pi_{j+1}) = \arg\min_q D_{KL}(q||\widehat\pi_{j+1}) - \mathbb{E}_{u \sim q}[\log \mathsf{I}(y_{j+1}^\dagger;u)]$$

We will define $q_j(\theta)$ recursively using a procedure, which depends on parameters $\theta = K$, where $K$ is the gain matrix. Then, the objective will be:
$$\min_\theta \frac{1}{J}\sum_{j=0}^J L_{j}(q_j(\theta))$$

#### 2.3.1 Data Assumption
We assume the physical dynamics $\Psi$, the initial condition $\mathbf{v}_0 \sim \mathcal{N}(m_0, C_0)$, and the noise covariances $\Sigma$ and $\Gamma$ are known. To generate training data, we simulate the system to obtain a set of ground truth trajectories $\{\mathbf{v}_j^\dagger\}_{j=1}^J$ and their corresponding noisy, sparse observations $\{\mathbf{y}_j^\dagger\}_{j=1}^J$, where $J$ denotes the total number of assimilation cycles in the training set.

#### 2.3.1 Define objective function

Consider Gaussian distributions $q_j(\theta)$ defined recursively given $K = \theta$:

**Prediction step**:

  \begin{align}
  \widehat{m}_{j+1} &= \Psi(m_j) \\
  \widehat{C}_{j+1} &= J_j C_j J_j^T + \Sigma, \qquad J_j = \text{ Jacobian of } \Psi
  \end{align}

**Analysis step**:

  \begin{align*}
  m_{j+1} &= \widehat{m}_{j+1} + K(y_{j+1}^\dagger - Hm_{j+1}) \\
  C_{j+1} &= (I - KH)\widehat{C}_{j+1} (I - KH)^T + K\Gamma K^T
  \end{align*}

Define: $\widehat\pi_{j+1} = \mathcal{N}(\widehat{m}_{j+1}, \widehat{C}_{j+1})$ and $q_{j+1}(K) = \mathcal{N}(m_{j+1}(K),C_{j+1}(K))$

**Goal**: Learn $K$ that yields close filters for all time $j$ by minimizing the variational inference objective for each step of filtering:
 $$L(K) := \frac{1}{J} \sum_{j=1}^J D_{KL}(q_{j+1}(K)||\widehat\pi_{j+1}) - \mathbb{E}_{u \sim q_{j+1}(K)}[\log \mathsf{I}(u;y_{j+1})]$$

### 2.3.2 Evaluation Metric

To evaluate the tracking accuracy of the learned gain matrix $K$ against the EnKF baseline, we employ the Root Mean Square Error (RMSE). This metric quantifies the divergence of the filtered state estimate from the ground truth trajectory. Let $\mathbf{v}_j^\dagger$ denote the true state and $\mathbf{m}_j$ the posterior mean estimate at time step $j$. Averaged over $J$ assimilation cycles and the system dimension $d=40$, the RMSE is defined as:

$$
\text{RMSE} = \sqrt{\frac{1}{J \cdot d} \sum_{j=1}^J \| \mathbf{m}_j - \mathbf{v}_j^\dagger \|_2^2}
$$

A lower RMSE indicates that the variational approximation successfully minimizes the information gap between the predicted and true distributions, maintaining the estimated trajectory close to the chaotic attractor.




In [ ]:
import jax
import jax.numpy as jnp
from jax import random, grad, jit, jacfwd, jacrev, lax
from jax.scipy.linalg import inv, eigh
from tqdm.auto import tqdm
from jax.tree_util import Partial
from functools import partial
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# ==========================================
# 1. Colab Parameters & Setup
# ==========================================

# @title System & Simulation Parameters
n = 40                  # @param {type:"integer"}
F = 8.0                 # @param {type:"number"}
dt = 0.03               # @param {type:"number"}
dt_obs = 0.15           # @param {type:"number"}
sigma_v = 0.01          # @param {type:"number"}
sigma_y = 1           # @param {type:"number"}
num_steps = 200         # @param {type:"integer"}

# @title Training/Optimization Parameters
J0 = 0                  # @param {type:"integer"}
N = 10                  # @param {type:"integer"}
alpha = 1e-5            # @param {type:"number"}
n_iters = 100           # @param {type:"integer"}

# --- Derived Parameters ---
observation_interval = int(dt_obs / dt)  # Steps per observation
print(f"Derived: Observation Interval = {observation_interval} steps")

# Model parameters
m0 = jnp.ones((n,))
C0 = jnp.eye(n) * 1.0
Q = jnp.eye(n) * (sigma_v ** 2)  # Process noise covariance
H = jnp.eye(n)                   # Full Observation
R = jnp.eye(n) * (sigma_y ** 2)  # Observation noise covariance

print(f"System Initialized: {n} States, Sigma_v={sigma_v}, Sigma_y={sigma_y}")
print(f"Training Settings: Alpha={alpha}, Iterations={n_iters}, MC Samples={N}")

# Learning Parameters
alpha = 1e-5        # Learning rate
n_iters = 100       # Iterations

print(f"System: dt={dt}, Interval={observation_interval}, Steps={num_steps}, Sigma_y={sigma_y}")
print(f"Training: Alpha={alpha}, Iters={n_iters}")

# ==========================================
# 2. Dynamics & Helpers
# ==========================================

@jit
def rk4_step(x, F, dt):
    f = lambda y: (jnp.roll(y, 1) - jnp.roll(y, -2)) * jnp.roll(y, -1) - y + F
    k1 = dt * f(x)
    k2 = dt * f(x + k1/2)
    k3 = dt * f(x + k2/2)
    k4 = dt * f(x + k3)
    return x + 1/6 * (k1 + 2 * k2 + 2 * k3 + k4)

class Lorenz96:
    def __init__(self, dt=0.01, F=8.0):
        self.dt = dt
        self.F = F

    def step(self, x):
        return rk4_step(x, self.F, self.dt)

@jit
def step_function(carry, input):
    key, x, observation_interval, H, Q, R, model_step, counter = carry
    n = len(x)
    key, subkey = random.split(key)
    x_j = model_step(x)

    # Conditional update logic
    def update_observation():
        x_noise = x_j + random.multivariate_normal(key, jnp.zeros(n), Q)
        obs_state = jnp.dot(H, x_noise)
        obs_noise = random.multivariate_normal(subkey, jnp.zeros(H.shape[0]), R)
        return x_noise, obs_state + obs_noise

    def no_update():
        return x_j, jnp.nan * jnp.ones(H.shape[0])

    x_j, obs = lax.cond(counter % observation_interval == 0,
                        update_observation,
                        no_update)
    counter += 1
    carry = (key, x_j, observation_interval, H, Q, R, model_step, counter)
    output = (x_j, obs)
    return carry, output

@partial(jit, static_argnums=(1, 2, 7))
def generate_true_states(key, num_steps, n, x0, H, Q, R, model_step, observation_interval):
    initial_carry = (key, x0, observation_interval, H, Q, R, model_step, 1)
    _, (xs, observations) = lax.scan(step_function, initial_carry, None, length=num_steps-1)

    key, subkey = random.split(key)
    initial_observation = H @ x0 + random.multivariate_normal(subkey, jnp.zeros(H.shape[0]), R)

    xs = jnp.vstack([x0[jnp.newaxis, :], xs])
    observations = jnp.vstack([initial_observation[jnp.newaxis, :], observations])
    return observations, xs

# Initialize
initial_state = random.normal(random.PRNGKey(42), (n,))
l96_model = Lorenz96(dt=dt, F=F)
l96_step = Partial(l96_model.step)

# Generate Data
key = random.PRNGKey(42)
observations, true_states = generate_true_states(key, num_steps, n, initial_state, H, Q, R, l96_step, observation_interval)
y = observations

# REMOVED: visualize_observations function

# ==========================================
# 3. Cost Function & Filtering
# ==========================================

jacobian_function = jacrev(l96_step, argnums=0)
jac_func = Partial(jacobian_function)

@jit
def KL_gaussian(n, m1, C1, m2, C2):
    C2_inv = inv(C2)
    # Added abs() and epsilon to log to prevent NaNs during rapid optimization
    vals_c2 = jnp.abs(eigh(C2)[0]) + 1e-10
    vals_c1 = jnp.abs(eigh(C1)[0]) + 1e-10
    log_det_ratio = (jnp.log(vals_c2).sum() - jnp.log(vals_c1).sum()).real
    return 0.5 * (log_det_ratio - n + jnp.trace(C2_inv @ C1) + ((m2 - m1).T @ C2_inv @ (m2 - m1)))

@jit
def log_likelihood(v, y, H, R, J, J0):
    def log_likelihood_j(_, v_y):
        v_j, y_j = v_y
        # Handle NaNs for sparse observations
        is_obs = ~jnp.isnan(y_j[0])
        error = jnp.where(is_obs, y_j - H @ v_j, 0.0)
        # Safe computation even with NaNs in y_j (masked by is_obs)
        ll = jnp.where(is_obs, error.T @ inv(R) @ error, 0.0)
        return _, ll

    _, lls = lax.scan(log_likelihood_j, None, (v, y))

    # Correct count of actual observations
    valid_count = J - jnp.sum(jnp.isnan(y[:, 0]))
    sum_ll = jnp.sum(lls)

    return -0.5 * sum_ll - 0.5 * (valid_count) * jnp.log(2 * jnp.pi) - 0.5 * (valid_count) * jnp.linalg.slogdet(R)[1]

@partial(jit, static_argnums=())
def KL_sum(m_preds, C_preds, m_updates, C_updates, n, state_transition_function, Q, key):
    def KL_j(_, m_C_y):
        m_pred, C_pred, m_update, C_update = m_C_y
        return _, KL_gaussian(n, m_update, C_update, m_pred, C_pred)
    _, mean_kls = lax.scan(KL_j, None, (m_preds, C_preds, m_updates, C_updates))
    kl_sum = jnp.sum(mean_kls)
    return kl_sum

@partial(jit, static_argnums=(4))
def apply_filtering_fixed_nonlinear(m0, C0, y, K, n, state_transition_function, jacobian_function, H, Q, R):
    partial_filter_step = lambda m_C_prev, y_curr: filter_step_nonlinear(m_C_prev, y_curr, K, n, state_transition_function, jacobian_function, H, Q, R)
    _, m_C = lax.scan(partial_filter_step, (m0, C0, m0, C0), y)
    return m_C

@partial(jit, static_argnums=(3))
def filter_step_nonlinear(m_C_prev, y_curr, K, n, state_transition_function, jacobian_function, H, Q, R):
    _, _, m_prev, C_prev = m_C_prev
    m_pred = state_transition_function(m_prev)
    F_jac = jacobian_function(m_prev)
    C_pred = F_jac @ C_prev @ F_jac.T + Q

    # Handle NaNs (Time sparsity)
    is_obs = ~jnp.isnan(y_curr[0])

    # Innov: if NaN, use 0 (masked later)
    innov = jnp.where(is_obs, y_curr, H @ m_pred) - (H @ m_pred)
    m_update = m_pred + jnp.where(is_obs, 1.0, 0.0) * (K @ innov)

    # Covariance update
    I_KH = jnp.eye(n) - K @ H
    C_update_obs = I_KH @ C_pred @ I_KH.T + K @ R @ K.T

    # If not observed, just C_pred
    C_update = (C_update_obs * is_obs) + (C_pred * (1.0 - is_obs))

    return (m_pred, C_pred, m_update, C_update), (m_pred, C_pred, m_update, C_update)

@partial(jit, static_argnums=(3, 10))
def var_cost(K, m0, C0, n, H, Q, R, y, key, num_steps, J0):
    m_preds, C_preds, m_updates, C_updates = apply_filtering_fixed_nonlinear(m0, C0, y, K, n, l96_step, jac_func, H, Q, R)
    key, *subkeys = random.split(key, num=N+1)

    kl_sum = KL_sum(m_preds, C_preds, m_updates, C_updates, n, l96_step, Q, key)

    # Use vmap for MC sampling speedup
    subkeys = jnp.vstack(subkeys)
    def inner_map(subkey):
        return log_likelihood(random.multivariate_normal(subkey, m_updates, C_updates), y, H, R, num_steps, J0)

    ll_samples = jax.vmap(inner_map)(subkeys)
    cost = kl_sum - jnp.mean(ll_samples)
    return cost

# ==========================================
# 4. Training Loop (ACCELERATED)
# ==========================================

@jit
def kalman_step(state, observation, params):
    m_prev, C_prev = state
    l96_step, jac_func, H, Q, R = params
    m_pred = l96_step(m_prev)
    F_jac = jac_func(m_prev)
    C_pred = F_jac @ C_prev @ F_jac.T + Q

    is_obs = ~jnp.isnan(observation[0])

    S = H @ C_pred @ H.T + R
    K_curr = C_pred @ H.T @ jnp.linalg.inv(S)

    K_use = K_curr * is_obs
    m_update = m_pred + K_use @ (jnp.where(is_obs, observation, H@m_pred) - H @ m_pred)
    C_update = (jnp.eye(n) - K_use @ H) @ C_pred

    return (m_update, C_update), (m_update, C_update, K_use)

@jit
def kalman_filter_process(m0, C0, observations):
    params = (l96_step, jac_func, H, Q, R)
    initial_state = (m0, C0)
    _, (m, C, K) = lax.scan(lambda state, obs: kalman_step(state, obs, params),
                            initial_state, observations)
    return m, C, K

print("Computing Baseline...")
base_m, base_C, base_K = kalman_filter_process(m0, C0, observations)
K_steady = jnp.mean(base_K[-50:, :, :], axis=0)

var_cost_grad = grad(var_cost, argnums=0)
K_opt = jnp.eye(n) * 0.4 # Keeping your initial guess

# --- ACCELERATION START ---
# We JIT compile the update step AND the evaluation calculation
# to avoid Python loop overhead.

@jit
def train_step(K, key):
    """Computes gradient and applies update in one JIT-compiled step."""
    grad_K = var_cost_grad(K, m0, C0, n, H, Q, R, y, key, num_steps, J0)
    K_new = K - alpha * grad_K
    return K_new

@jit
def evaluate_step(K, true_states, base_m, base_C):
    """Computes error and KL divergence in JIT."""
    _, _, predicted_states, covariances = apply_filtering_fixed_nonlinear(m0, C0, y, K, n, l96_step, jac_func, H, Q, R)

    # MSE
    prediction_error = jnp.mean(jnp.mean((predicted_states - true_states)**2, axis=1))

    # KL Div to Baseline (Vectorized)
    def compute_kl(p_m, p_c, b_m, b_c):
        return KL_gaussian(n, p_m, p_c, b_m, b_c)

    kl_divs = jax.vmap(compute_kl)(predicted_states, covariances, base_m, base_C)
    mean_kl = jnp.mean(kl_divs)

    return prediction_error, mean_kl

# --- ACCELERATION END ---

prediction_errors = []
true_div = []

print(f"Starting Training ({n_iters} iterations)...")
for i in tqdm(range(n_iters)):
    key, subkey = random.split(key)

    # 1. Update (JIT accelerated)
    K_opt = train_step(K_opt, subkey)

    # 2. Evaluate (JIT accelerated)
    # Note: Evaluating every step is still costly, but JIT helps.
    err, kl = evaluate_step(K_opt, true_states, base_m, base_C)

    prediction_errors.append(err)
    true_div.append(kl)

# ==========================================
# 5. Plotting
# ==========================================

def plot_optimization_results(prediction_errors, true_div, n_iters, scaling=1.3, max_n_locator=5):
    fig, (ax1, ax3) = plt.subplots(figsize=(10, 4), ncols=2)

    # K norms and KL Divergence
    ax1.set_xlabel('Iteration', fontsize=14*scaling)
    ax1.tick_params(axis='y', labelsize=12*scaling)
    ax1.tick_params(axis='x', labelsize=12*scaling)
    ax1.yaxis.set_major_locator(plt.MaxNLocator(max_n_locator))
    ax1.set_ylabel('KL divergence to true filter', fontsize=14*scaling)
    ax1.plot(range(1, n_iters+1), true_div[:n_iters], label='KL divergence to true filter')

    # MSE from True States
    ax3.plot(range(1, n_iters+1), prediction_errors[:n_iters])
    ax3.set_xlabel("Iteration", fontsize=14*scaling)
    ax3.set_ylabel("Prediction error (MSE)", fontsize=14*scaling)
    ax3.tick_params(axis='x', labelsize=12*scaling)
    ax3.tick_params(axis='y', labelsize=12*scaling)
    ax3.yaxis.set_major_locator(plt.MaxNLocator(max_n_locator))

    plt.tight_layout()
    plt.show()

plot_optimization_results(prediction_errors, true_div, n_iters, scaling=1.3, max_n_locator=3)

import numpy as np

def plot_k_matrices(K_steady, K_opt, scaling=1.2, max_n_locator=5):
    # 1. Determine shared color scale limits
    vmin = min(np.min(K_steady), np.min(K_opt))
    vmax = max(np.max(K_steady), np.max(K_opt))

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(figsize=(10, 5), ncols=2, constrained_layout=True)

    # Plot K_steady
    c1 = ax1.pcolormesh(K_steady, cmap='RdBu_r', vmin=vmin, vmax=vmax)
    ax1.set_title('$K_\\mathrm{steady}$ (Baseline)', fontsize=14*scaling)
    ax1.invert_yaxis()
    ax1.tick_params(axis='both', labelsize=12*scaling)
    ax1.yaxis.set_major_locator(plt.MaxNLocator(max_n_locator))

    # Plot K_opt
    c2 = ax2.pcolormesh(K_opt, cmap='RdBu_r', vmin=vmin, vmax=vmax)
    ax2.set_title('$K_\\mathrm{opt}$ (Learned)', fontsize=14*scaling)
    ax2.invert_yaxis()
    ax2.tick_params(axis='both', labelsize=12*scaling)
    ax2.yaxis.set_major_locator(plt.MaxNLocator(max_n_locator))

    # Add Shared Colorbar
    cbar = fig.colorbar(c2, ax=[ax1, ax2], orientation='vertical', fraction=0.05, pad=0.02)
    cbar.ax.tick_params(labelsize=12*scaling)
    cbar.set_label('Gain Value', fontsize=12*scaling)

    plt.show()

# Plot the matrices
plot_k_matrices(K_steady, K_opt, scaling=1.2)

# --- Save Learned Trajectory Mean ---

print("Generating final trajectory with K_opt...")
# Run the filter one last time with the optimized K
_, _, learned_traj_mean, _ = apply_filtering_fixed_nonlinear(m0, C0, y, K_opt, n, l96_step, jac_func, H, Q, R)

# Save to file
save_path = 'learned_traj_mean.npy'
np.save(save_path, learned_traj_mean)
print(f"Learned trajectory mean saved to: {save_path}")
print(f"Shape: {learned_traj_mean.shape}")

In [ ]:
# ==========================================
# 2.3.2 Evaluation: EnKF Baseline & RMSE Comparison
# ==========================================

# @title EnKF Parameters
N_ens = 40              # @param {type:"integer"}
inflation = 1.05        # @param {type:"number"}

# ---------------------------------------------------------
# 1. EnKF Implementation (Handles Sparse Observations)
# ---------------------------------------------------------

@partial(jit, static_argnums=(0, 1))
def enkf_step_fn(N_ens, infl, carry, y_curr):
    """
    Single step of the Ensemble Kalman Filter.
    """
    key, ens = carry
    key, subkey_q, subkey_r = random.split(key, 3)

    # --- Forecast Step ---
    ens_pred = jax.vmap(l96_step)(ens)
    q_noise = random.multivariate_normal(subkey_q, jnp.zeros(n), Q, shape=(N_ens,))
    ens_pred = ens_pred + q_noise

    # --- Analysis Step ---
    is_obs = ~jnp.isnan(y_curr[0])

    def update(ens_p):
        r_noise = random.multivariate_normal(subkey_r, jnp.zeros(n), R, shape=(N_ens,))
        y_perturbed = y_curr + r_noise

        h_ens = ens_p
        x_bar = jnp.mean(ens_p, axis=0)
        h_bar = jnp.mean(h_ens, axis=0)

        X_prime = ens_p - x_bar
        H_prime = h_ens - h_bar

        P_xy = (X_prime.T @ H_prime) / (N_ens - 1)
        P_yy = (H_prime.T @ H_prime) / (N_ens - 1) + R

        K_gain = P_xy @ jnp.linalg.inv(P_yy)

        innov = y_perturbed - h_ens
        ens_ana = ens_p + (K_gain @ innov.T).T

        ens_mean_ana = jnp.mean(ens_ana, axis=0)
        ens_ana = ens_mean_ana + infl * (ens_ana - ens_mean_ana)
        return ens_ana

    def no_update(ens_p):
        return ens_p

    ens_updated = lax.cond(is_obs, update, no_update, ens_pred)
    step_mean = jnp.mean(ens_updated, axis=0)

    return (key, ens_updated), step_mean

def run_enkf(N_ens, infl, y_seq, m0, C0, key):
    key, subkey = random.split(key)
    # Initialize ensemble around m0
    ens_0 = random.multivariate_normal(subkey, m0, C0, shape=(N_ens,))

    scan_fn = partial(enkf_step_fn, N_ens, infl)
    _, enkf_means = lax.scan(scan_fn, (key, ens_0), y_seq)

    return enkf_means

# ---------------------------------------------------------
# 2. Execution & RMSE Comparison
# ---------------------------------------------------------

print(f"Running EnKF with N={N_ens} ensemble members...")
enkf_key = random.PRNGKey(101)
# Generate EnKF trajectory
enkf_traj_mean = run_enkf(N_ens, inflation, y, m0, C0, enkf_key)

# RMSE Calculation Function
def calculate_rmse(estimated_traj, true_traj):
    """
    Computes RMSE averaged over time (J) and dimensions (d).
    RMSE = sqrt( 1/(J*d) * sum( ||m_j - v_j||^2 ) )
    """
    # Ensure shapes match before calculation
    # Truncate to the minimum length to avoid broadcasting errors
    min_len = min(len(estimated_traj), len(true_traj))
    est = estimated_traj[:min_len]
    tru = true_traj[:min_len]

    sq_diff = (est - tru) ** 2
    mean_sq_error = jnp.mean(sq_diff)
    return jnp.sqrt(mean_sq_error)

# Calculate RMSEs (Corrected: No [1:] slicing)
rmse_learned = calculate_rmse(learned_traj_mean, true_states)
rmse_enkf = calculate_rmse(enkf_traj_mean, true_states)

# ---------------------------------------------------------
# 3. Results Output
# ---------------------------------------------------------

print("\n" + "="*40)
print("### 2.3.2 Evaluation Results (RMSE)")
print("="*40)
print(f"Learned Gain (VI) RMSE : {rmse_learned:.5f}")
print(f"EnKF (N={N_ens}) RMSE      : {rmse_enkf:.5f}")
print("-" * 40)

diff = rmse_enkf - rmse_learned
if diff > 0:
    print(f">> Result: Learned Gain outperforms EnKF by {diff:.5f}")
else:
    print(f">> Result: EnKF outperforms Learned Gain by {-diff:.5f}")
print("="*40)

# Plotting
plt.figure(figsize=(12, 5))
dim_to_plot = 0
# Plot a subset of steps for clarity
steps_to_show = 100
t_axis = jnp.arange(steps_to_show)

plt.plot(t_axis, true_states[:steps_to_show, dim_to_plot], 'k-', lw=1.5, alpha=0.6, label='True State')
plt.plot(t_axis, learned_traj_mean[:steps_to_show, dim_to_plot], 'r--', lw=1.5, label=f'Learned (RMSE={rmse_learned:.3f})')
plt.plot(t_axis, enkf_traj_mean[:steps_to_show, dim_to_plot], 'g:', lw=2, label=f'EnKF (RMSE={rmse_enkf:.3f})')

plt.title(f"Trajectory Reconstruction (Dim {dim_to_plot}, First {steps_to_show} Steps)")
plt.xlabel("Time Step")
plt.ylabel("State Value")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2.4 Machine Learning Enhanced EnKF Strategies

We explore two paradigms for integrating machine learning into the data assimilation cycle. Instead of relying on manual tuning or empirical approximations, we parameterize critical components of the filter and optimize them through a data-driven framework.

#### **Data Assumption**
We assume the physical dynamics $\Psi$, the initial condition $\mathbf{v}_0 \sim \mathcal{N}(m_0, C_0)$, and the noise covariances $\Sigma$ and $\Gamma$ are known. To generate training data, we simulate the system to obtain a set of ground truth trajectories $\{\mathbf{v}_j^\dagger\}_{j=1}^J$ and their corresponding noisy, sparse observations $\{\mathbf{y}_j^\dagger\}_{j=1}^J$, where $J$ denotes the total number of assimilation cycles in the training set.

---

### 2.4.1 Learning Regularization Parameters
In this approach, we treat the inflation rate $\alpha$ and the localization radius $r$ as learnable parameters. While these are traditionally fixed constants, we represent them as optimizable variables $\theta_{\text{reg}} = \{\alpha, r\}$. This allows the regularization to be optimized directly for the 40-dimensional Lorenz-96 space, providing the necessary ensemble spread and spatial filtering to prevent filter divergence.

### 2.4.2 Neural Kalman Gain Operator
This method replaces the empirical Kalman Gain calculation with a learned neural operator $\mathbf{K}_\theta$. The analysis update for each ensemble member is reformulated as:

$$
\mathbf{v}_j^{(n)} = \hat{\mathbf{v}}_j^{(n)} + \mathbf{K}_\theta(\dots) \underbrace{\left( \mathbf{y}^\dagger_j - h(\hat{\mathbf{v}}_j^{(n)}) - \eta_j^{(n)} \right)}_{\text{Innovation}}
$$

The neural gain $\mathbf{K}_\theta$ is designed to capture complex, potentially non-linear mappings. Its output is conditioned on the following features:
* **Ensemble Statistics:** The predictive mean $\bar{\mathbf{v}}_j$ and the predictive covariance $\hat{\mathbf{C}}_j$.
* **Local State Information:** The individual forecast member $\hat{\mathbf{v}}_j^{(n)}$.
* **Observation Data:** The actual measurement $\mathbf{y}_j^\dagger$ and the resulting innovation vector.

---

### 2.4.3 Training Objective (Loss Function)
The parameters $\theta$ are optimized by minimizing the discrepancy between the filter's estimation and the known ground truth. We define the loss function as the Mean Squared Error (MSE) or Normalized MSE between the analysis ensemble mean and the true state:

$$
\mathcal{L}(\theta) = \frac{1}{J} \sum_{j=1}^J \| \bar{\mathbf{v}}_j^{\text{ana}}(\theta) - \mathbf{v}_j^\dagger \|^2
$$

where $\bar{\mathbf{v}}_j^{\text{ana}}(\theta)$ is the mean of the ensemble after the ML-augmented analysis step. This objective directly rewards the model for producing the most accurate state estimate across the training trajectory.

### 2.4.4 Evaluation Metrics
To compare the performance of the ML-enhanced filters against the baseline EnKF, we use the Root Mean Squared Error (RMSE) calculated over a separate, unseen test trajectory:

$$
\text{RMSE} = \sqrt{\frac{1}{J_{\text{test}} \cdot 40} \sum_{j=1}^{J_{\text{test}}} \sum_{k=1}^{40} (v_{j,k}^{\text{est}} - v_{j,k}^\dagger)^2}
$$

A lower RMSE indicates that the machine learning model has successfully learned to compensate for the limitations of the standard ensemble-based update, such as sampling errors or suboptimal localization.

In [ ]:
# ==========================================
# 1. Colab Form Configuration
# ==========================================
# @title Lorenz-96 Data Generation Parameters

# @markdown **Time Stepping**
delta_t = 0.15 # @param {type:"number"}
rk4_steps = 5 # @param {type:"integer"}

# @markdown **Noise Levels**
sigma_y = 1.0 # @param {type:"number"}
sigma_v = 0.01 # @param {type:"number"}

# @markdown **Trajectory Settings**
traj_steps = 60 # @param {type:"integer"}
traj_nums = 250 # @param {type:"integer"}
burn_in_steps = 1000 # @param {type:"integer"}

import jax
import jax.numpy as jnp
from jax import random, jit, vmap, lax
from functools import partial
import matplotlib.pyplot as plt
import numpy as np

# Constants (Fixed for Lorenz-96 standard case)
DIM = 40
F_FORCE = 8.0
DT_INT = delta_t / rk4_steps
# Full observation: Observe all state dimensions
H_IDX = jnp.arange(DIM)

# ==========================================
# 2. Lorenz-96 Dynamics & Solver
# ==========================================

@jit
def lorenz96_deriv(v):
    """Lorenz-96 equations with cyclic boundaries."""
    return (jnp.roll(v, -1) - jnp.roll(v, 2)) * jnp.roll(v, 1) - v + F_FORCE

@jit
def rk4_step(v, dt):
    """Standard RK4 integration step."""
    k1 = lorenz96_deriv(v)
    k2 = lorenz96_deriv(v + 0.5 * dt * k1)
    k3 = lorenz96_deriv(v + 0.5 * dt * k2)
    k4 = lorenz96_deriv(v + dt * k3)
    return v + (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)

@partial(jit, static_argnums=(1,))
def integrate_window(v_init, steps, dt):
    """Integrate for one assimilation window (Delta t)."""
    def body_fn(v, _):
        return rk4_step(v, dt), None
    v_final, _ = lax.scan(body_fn, v_init, None, length=steps)
    return v_final

# ==========================================
# 3. Data Generation (Vmapped)
# ==========================================

@partial(jit, static_argnums=(1, 2, 3))
def generate_single_traj(key, b_steps, t_steps, r_steps):
    """Generates one truth trajectory and corresponding observations."""
    k_init, k_dyn, k_obs = random.split(key, 3)

    # 1. Burn-in to find v0 on attractor
    v_start = random.normal(k_init, (DIM,))
    def burn_in_step(v_carry, _):
        return rk4_step(v_carry, 0.01), None
    v0, _ = lax.scan(burn_in_step, v_start, None, length=b_steps)

    # 2. Generate trajectory with dynamic noise
    def scan_fn(v_prev, k_step):
        # Forecast for one window
        v_next_clean = integrate_window(v_prev, r_steps, DT_INT)
        # Add dynamic noise
        v_next = v_next_clean + sigma_v * random.normal(k_step, (DIM,))
        return v_next, v_next

    keys_dyn = random.split(k_dyn, t_steps)
    _, v_traj = lax.scan(scan_fn, v0, keys_dyn)
    v_traj = jnp.concatenate([v0[None, :], v_traj], axis=0)

    # 3. Generate observations (full)
    y_clean = v_traj[1:, H_IDX]
    y_obs = y_clean + sigma_y * random.normal(k_obs, y_clean.shape)

    return v_traj, y_obs

print(f"Generating {traj_nums} trajectories parallelly...")
keys = random.split(random.PRNGKey(42), traj_nums)
all_v, all_y = vmap(lambda k: generate_single_traj(k, burn_in_steps, traj_steps, rk4_steps))(keys)

# Split 80/20
split_idx = int(0.8 * traj_nums)
train_v, test_v = all_v[:split_idx], all_v[split_idx:]
train_y, test_y = all_y[:split_idx], all_y[split_idx:]

# Save
jnp.savez("lorenz96_data.npz",
          train_v=train_v, train_y=train_y,
          test_v=test_v, test_y=test_y)
print(f"Data saved: {train_v.shape[0]} train trajs, {test_v.shape[0]} test trajs.")

# ==========================================
# 4. Visualizations
# ==========================================

# 4.1 Visualization: GC Function
def gc_function(d, r):
    x = jnp.abs(d) / r
    return jnp.where(x < 1.0,
                     1 - 5/3*x**2 + 5/8*x**3 + 1/2*x**4 - 1/4*x**5,
                     jnp.where(x < 2.0,
                               4 - 5*x + 5/3*x**2 + 5/8*x**3 - 1/2*x**4 + 1/12*x**5 - 2/(3*x),
                               0.0))

plt.figure(figsize=(8, 4))
d_vals = jnp.linspace(0, 12, 200)
for r_val in [2.0, 4.0, 6.0]:
    plt.plot(d_vals, gc_function(d_vals, r_val), label=f"r={r_val}", lw=2)
plt.title("Gaspari-Cohn Localization Weights")
plt.xlabel("Distance"); plt.ylabel("Weight")
plt.legend(); plt.grid(True, alpha=0.3)
plt.show()

# 4.2 Visualization: Lorenz-96 Trajectory Heatmap
plt.figure(figsize=(12, 5))
sample_traj = all_v[0, 1:]
plt.imshow(sample_traj.T, aspect='auto', cmap='coolwarm', origin='lower')
plt.colorbar(label="State Value")
plt.title(f"Lorenz-96 Ground Truth (First {traj_steps} Cycles)")
plt.xlabel("Time Step (Assimilation Cycle)"); plt.ylabel("Dimension Index")
plt.tight_layout()
plt.grid(False)
plt.show()

In [ ]:
# ==========================================
# 1. Training & Evaluation Configuration
# ==========================================
# @title EnKF Hyperparameter Optimization (Robust Version)

optimizer_choice = "Adam" # @param ["L-BFGS-B", "Adam", "AdamW"]
learning_rate = 0.0001 # @param {type:"number"}
weight_decay = 0.01 # @param {type:"number"}
training_iterations = 500 # @param {type:"integer"}
train_ens_size_val = 100 # @param {type:"integer"}

# @markdown ---
# Test settings
test_ens_sizes = [10, 50, 100, 200]

import jax
import jax.numpy as jnp
from jax import random, jit, vmap, value_and_grad, lax
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize
from functools import partial
from tqdm.auto import tqdm

# Precompute cyclic distance matrix (State Dim 40 x Obs Dim 40)
dist_mat = jnp.abs(jnp.arange(40)[:, None] - H_IDX[None, :])
dist_mat = jnp.minimum(dist_mat, 40 - dist_mat)

# ==========================================
# 2. Core Assimilation Logic
# ==========================================

@jit
def gc_function(d, r):
    """Gaspari-Cohn localization kernel."""
    x = d / r
    return jnp.where(x < 1.0,
                     1 - 5/3*x**2 + 5/8*x**3 + 1/2*x**4 - 1/4*x**5,
                     jnp.where(x < 2.0,
                               4 - 5*x + 5/3*x**2 + 5/8*x**3 - 1/2*x**4 + 1/12*x**5 - 2/(3*x),
                               0.0))

@jit
def enkf_cycle_step(carry, inputs, alpha, r, ens_size):
    """Single EnKF assimilation cycle."""
    ens, k_step = carry
    y_obs, _ = inputs
    k1, k2, k3 = random.split(k_step, 3)

    # 1. Forecast: Integrate members for one window + dynamic noise
    ens_f = vmap(lambda v: integrate_window(v, rk4_steps, DT_INT))(ens)
    ens_f += sigma_v * random.normal(k1, ens_f.shape)

    # 2. Localized Analysis
    ens_mean = jnp.mean(ens_f, axis=0)
    anom = ens_f - ens_mean
    h_ens = ens_f[:, H_IDX]
    h_anom = h_ens - jnp.mean(h_ens, axis=0)

    C_vh = (1.0 / (ens_size - 1)) * (anom.T @ h_anom)
    C_hh = (1.0 / (ens_size - 1)) * (h_anom.T @ h_anom)

    L = gc_function(dist_mat, r)
    C_vh_loc = L * C_vh

    # CHANGED: 10 -> 40 (Full observation dimension)
    R_mat = (sigma_y**2) * jnp.eye(40)
    K = C_vh_loc @ jnp.linalg.inv(C_hh + R_mat)

    eta = sigma_y * random.normal(k2, h_ens.shape)
    innovation = y_obs[None, :] - h_ens - eta
    ens_a = ens_f + (innovation @ K.T)

    # 3. Post-Analysis Inflation
    ens_a_mean = jnp.mean(ens_a, axis=0)
    ens_a = ens_a_mean + alpha * (ens_a - ens_a_mean)

    return (ens_a, k3), ens_a_mean

@partial(jit, static_argnums=(3,))
def assimilate_l96_trajectory(params, v_true_traj, y_obs_traj, ens_size, key):
    """Full DA sequence for a Lorenz-96 trajectory."""
    alpha, r = params
    k_init, k_cycle = random.split(key)
    ens_init = v_true_traj[0] + random.normal(k_init, (ens_size, DIM))

    _, mean_traj_out = lax.scan(
        lambda c, i: enkf_cycle_step(c, i, alpha, r, ens_size),
        (ens_init, k_cycle),
        (y_obs_traj, v_true_traj[1:])
    )
    return mean_traj_out

# ==========================================
# 3. Training Logic (Direct & Fail-Fast)
# ==========================================

truncation_steps = 10  # Window size for backprop

@jit
def sequence_loss(params, start_v, obs_seq, truth_seq, key):
    """
    Computes pure MSE loss. No penalties, no masking.
    """
    alpha, r = params
    ens_size = train_ens_size_val

    # Initialize ensemble
    k_init, k_scan = random.split(key)
    ens_init = start_v + random.normal(k_init, (ens_size, DIM))

    def scan_body(carry, inputs):
        ens_prev, k = carry
        y, _ = inputs
        (ens_next, k_next), mean_next = enkf_cycle_step((ens_prev, k), (y, None), alpha, r, ens_size)
        return (ens_next, k_next), mean_next

    _, mean_traj = lax.scan(
        scan_body,
        (ens_init, k_scan),
        (obs_seq, truth_seq),
        length=truncation_steps
    )

    # Direct MSE calculation. If this explodes, we let it explode.
    diff = mean_traj - truth_seq
    mse = jnp.mean(diff**2)
    return mse

@jit
def train_step(params, m, v, batch_idx, key, i):
    """
    Performs one update step. Returns valid=False if NaNs occur.
    """

    def batch_loss_fn(p):
        # Select random windows for the batch
        # Note: We use a simple hash of index+iter for randomness to stay JIT-compatible
        def single_traj_loss(b_i):
            k_local = random.fold_in(key, b_i)
            t_start = random.randint(k_local, shape=(), minval=0, maxval=traj_steps - truncation_steps)

            # Use dynamic_slice to handle random start index
            curr_v = train_v[b_i]
            curr_y = train_y[b_i]

            v_start = lax.dynamic_slice(curr_v, (t_start, 0), (1, DIM))[0]
            truth_seq = lax.dynamic_slice(curr_v, (t_start + 1, 0), (truncation_steps, DIM))
            # CHANGED: 10 -> 40 (Full observation dimension)
            obs_seq = lax.dynamic_slice(curr_y, (t_start, 0), (truncation_steps, 40))

            return sequence_loss(p, v_start, obs_seq, truth_seq, k_local)

        # Batch loss
        losses = vmap(single_traj_loss)(jnp.arange(32)) # Fixed batch size 32
        return jnp.mean(losses)

    # Compute Gradients
    loss_val, grads = value_and_grad(batch_loss_fn)(params)

    # Check for NaNs
    is_nan = jnp.any(jnp.isnan(grads)) | jnp.isnan(loss_val) | jnp.any(jnp.isinf(grads))

    # Update logic (only if not NaN)
    # If NaN, we return old params and set valid=False

    # Clip grads
    grads = jnp.clip(grads, -1.0, 1.0)

    m_new = 0.9 * m + 0.1 * grads
    v_new = 0.999 * v + 0.001 * (grads**2)
    m_hat = m_new / (1 - 0.9**(i+1))
    v_hat = v_new / (1 - 0.999**(i+1))

    update = learning_rate * m_hat / (jnp.sqrt(v_hat) + 1e-8)
    params_new = params - update

    # Constraints
    params_new = jnp.array([
        jnp.clip(params_new[0], 1.001, 1.5),
        jnp.clip(params_new[1], 1.0, 20.0)
    ])

    # If is_nan, keep old params
    final_params = jnp.where(is_nan, params, params_new)
    final_m = jnp.where(is_nan, m, m_new)
    final_v = jnp.where(is_nan, v, v_new)

    return final_params, final_m, final_v, loss_val, is_nan

# ==========================================
# Execution Loop
# ==========================================

# Reset Parameters
params = jnp.array([1.05, 5.0])
m, v = jnp.zeros(2), jnp.zeros(2)

print(f"Starting Training (Window={truncation_steps})...")
pbar = tqdm(range(training_iterations), desc="Training")

for i in pbar:
    key = random.PRNGKey(i)
    params, m, v, loss_val, is_nan_flag = train_step(params, m, v, None, key, i)

    if is_nan_flag:
        print(f"\n[CRITICAL] Training diverged at iteration {i}!")
        print(f"Last Loss: {loss_val}")
        print(f"Current Params: Alpha={params[0]:.4f}, R={params[1]:.4f}")
        print("Stopping training immediately.")
        break

    pbar.set_postfix({"loss": f"{loss_val:.2f}", "α": f"{params[0]:.3f}", "r": f"{params[1]:.3f}"})

opt_alpha, opt_r = params
print(f"\nFinal Parameters -> Alpha: {opt_alpha:.4f}, Radius: {opt_r:.4f}")

# ==========================================
# 4. Comparative Evaluation
# ==========================================

def run_test_evaluation(a_learned, r_learned):
    n_test = 50
    print("\n" + "="*95)
    print(f"{'Ens Size':<10} | {'Baseline RMSE':<15} | {'Base NaN%':<10} | {'Learned RMSE':<15} | {'Learned NaN%':<10}")
    print("-" * 95)

    test_results = {}
    for sz in test_ens_sizes:
        def eval_single(i, a, r):
            m_traj = assimilate_l96_trajectory([a, r], test_v[i], test_y[i], sz, random.PRNGKey(i+5000))
            rmse = jnp.sqrt(jnp.mean((m_traj - test_v[i, 1:])**2))
            return rmse, jnp.isnan(rmse), m_traj

        # Baseline: No inflation (1.0), Moderate Radius (4.0)
        b_rmse_list, b_nan_list, b_trajs = vmap(lambda i: eval_single(i, 1.0, 4.0))(jnp.arange(n_test))
        l_rmse_list, l_nan_list, l_trajs = vmap(lambda i: eval_single(i, a_learned, r_learned))(jnp.arange(n_test))

        def compute_stats(rmses, nans):
            valid = ~nans
            avg = jnp.sum(jnp.where(valid, rmses, 0.0)) / jnp.maximum(jnp.sum(valid), 1)
            percent = (jnp.sum(nans) / n_test) * 100
            return avg, percent

        b_avg, b_nan_p = compute_stats(b_rmse_list, b_nan_list)
        l_avg, l_nan_p = compute_stats(l_rmse_list, l_nan_list)
        print(f"{sz:<10d} | {b_avg:<15.4f} | {b_nan_p:<9.1f}% | {l_avg:<15.4f} | {l_nan_p:<10.1f}%")

        test_results[sz] = {"b_trajs": b_trajs, "l_trajs": l_trajs, "l_nans": l_nan_list}
    print("="*95)
    return test_results

test_metrics = run_test_evaluation(opt_alpha, opt_r)

# ==========================================
# 5. Robust Visualization
# ==========================================

def find_stable_index(results_data):
    for sz in reversed(test_ens_sizes):
        l_nans = results_data[sz]["l_nans"]
        valid_idxs = jnp.where(~l_nans)[0]
        if len(valid_idxs) > 0:
            return int(valid_idxs[0]), sz
    return None, None

plot_idx, plot_ens = find_stable_index(test_metrics)

if plot_idx is not None:
    print(f"\nVisualizing Test Index: {plot_idx} (Using stable data from Ens={plot_ens})")
    v_true = test_v[plot_idx, 1:]
    b_plot = test_metrics[plot_ens]["b_trajs"][plot_idx]
    l_plot = test_metrics[plot_ens]["l_trajs"][plot_idx]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    titles = ["Ground Truth", f"Baseline (Ens={plot_ens})", f"Learned (Ens={plot_ens})"]
    v_min, v_max = v_true.min(), v_true.max()

    for i, data_mat in enumerate([v_true, b_plot, l_plot]):
        im = axes[i].imshow(data_mat.T, aspect='auto', cmap='coolwarm', origin='lower', vmin=v_min, vmax=v_max)
        axes[i].set_title(titles[i])
        axes[i].set_xlabel("Cycle")
        axes[i].set_ylabel("Dimension")
        plt.colorbar(im, ax=axes[i])
    plt.tight_layout(); plt.show()
else:
    print("\n[Critical Warning]: All tested configurations resulted in filter divergence (NaN).")